In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import loguniform
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from safetensors.torch import load_file
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
import os
os.environ["WANDB_DISABLED"] = "true"

# 데이터 로드 및 준비

In [2]:
# 데이터 불러오기
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
train_df.head()

,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2


In [3]:
submission_df = pd.read_csv("../data/sample_submission.csv")

In [4]:
submission_df.info()
submission_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        1780 non-null   object
 1   answer_0  1780 non-null   int64 
 2   answer_1  1780 non-null   int64 
 3   answer_2  1780 non-null   int64 
 4   answer_3  1780 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 69.7+ KB


,ID,answer_0,answer_1,answer_2,answer_3
0,TEST_0000,0,1,2,3
1,TEST_0001,0,1,2,3
2,TEST_0002,0,1,2,3
3,TEST_0003,0,1,2,3
4,TEST_0004,0,1,2,3


In [5]:
# 텍스트 정제
def clean_text(text):
  # 특수문자 제거
  text = re.sub(r'[^\w\s]', '', text)
  # 소문자 변환: 한글에는 무의미
  text = text.lower()
  # 불필요한 공백 제거
  text = ' '.join(text.split())
  return text

In [6]:
# 텍스트 정제
for i in range(4):
    train_df[f'sentence_{i}'] = train_df[f'sentence_{i}'].apply(clean_text)
    test_df[f'sentence_{i}'] = test_df[f'sentence_{i}'].apply(clean_text)

In [7]:
def make_labels(df):
    # answer_0 ~ answer_3 → [문장0은 몇 번째, 문장1은 몇 번째, ...]
    answers = df[[f'answer_{i}' for i in range(4)]].values
    labels = []
    for row in answers:
        label = [0]*4
        for pos, sent_idx in enumerate(row):
            label[sent_idx] = pos
        labels.append(label)
    return np.array(labels)

# Dataset 클래스
 4개의 문장을 [SEP]로 묶어서 BERT에 넣을 수 있게 바꿔줌

In [8]:
# ✅ 1. 데이터셋 클래스
class GlobalOrderDataset(Dataset):
    def __init__(self, df, tokenizer, labels=None, max_length=256):
        self.sentences = df[[f'sentence_{i}' for i in range(4)]].values
        self.tokenizer = tokenizer
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sents = self.sentences[idx]
        text = '[CLS] ' + ' [SEP] '.join(sents) + ' [SEP]'
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {k: v.squeeze(0) for k, v in encoding.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
            return item

# Model 클래스
- AutoModel (예: Roberta) 사용
- 문장 4개를 넣었을 때 그 순서를 예측
- 출력은 [batch, 4, 4] 크기의 행렬 → 각 문장이 어떤 위치에 있어야 하는지 예측

In [18]:
class GlobalOrderModel(nn.Module):
    def __init__(self, model_name='klue/roberta-large', tokenizer=None):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.tokenizer = tokenizer
        hidden_size = self.bert.config.hidden_size
        
        # 기본 분류기 구조
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 4 * 4)  # 4문장 * 4 클래스
        )
        
    def forward(self, input_ids, attention_mask, labels=None):
        # BERT 출력
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0]  # [CLS] 토큰만 사용
        
        # 분류
        logits = self.classifier(pooled)
        
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, 4), labels.view(-1))
            return {"loss": loss, "logits": logits.view(-1, 4, 4)}
        else:
            return {"logits": logits.view(-1, 4, 4)}

In [10]:
def compute_metrics(eval_pred):
    try:
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=2)
        sentence_accuracy = (preds == labels).mean()
        full_order_accuracy = (preds == labels).all(axis=1).mean()
        return {
            "sentence_accuracy": sentence_accuracy,
            "full_order_accuracy": full_order_accuracy
        }
    except Exception as e:
        print(f"❌ compute_metrics 내부 오류: {e}")
        return {}


In [11]:
# ✅ train/val 분리 (20% → 검증에 사용)
train_df_split, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# ✅ 라벨 생성
train_labels = make_labels(train_df_split)
val_labels = make_labels(val_df)

In [19]:
# ✅ RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

# 🔹 라벨 생성
labels = make_labels(train_df_split)

# 모델 초기화
model = GlobalOrderModel(
    model_name="klue/roberta-large",
    tokenizer=tokenizer  # tokenizer 전달
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # ✅ GPU 사용 여부 확인
model.to(device)  # ✅ 모델을 해당 디바이스로 이동

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GlobalOrderModel(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (L

In [13]:
# 🔹 학습 데이터셋 생성
train_dataset = GlobalOrderDataset(train_df_split, tokenizer, labels=train_labels)
val_dataset = GlobalOrderDataset(val_df, tokenizer, labels=val_labels)

In [14]:
train_dataset[0]

{'input_ids': tensor([    0,     0,  3857,  7285,  4206,  3794,  3747,  2170,  3844,  2530,
          6233,  2525,  3773,  2079,  4901,  2047,  2145,  5068, 11604,  7594,
          2897,  2062,     2,  7655,  7246,  2079, 18309,  2145,  3828,  2125,
          3979,  2069,  4146,  2085,   904,  3892, 31221,  4253, 25052,  2116,
          5588,  2125, 28674,     2,  3983,  7655,  4253, 25052,  2259,  3857,
          2145,  2079,  4203,  2069,  3644,  3940,  4021,  2651,  1295,  1513,
          2062,     2,  1504,  3747, 27135,  3857,  3844,  2259,  5767,  2047,
          2069,  4392,  2088,  3634,  3647,  2079,  4301,  2138,  4523,  2205,
          2259,  3748,  2470,  4008,  2069,  3605,     2,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [15]:
val_dataset[0]

{'input_ids': tensor([    0,     0, 13025,  4331,  4119,  4525,  2170,  2259,  4262, 31221,
          4254,  6087,  2116,  5588,  2125, 28674,     2,  4331,  5189,  2079,
          4040,  2470, 10291,  2069,  3627, 13149,  2267,  3726,  2052,  3677,
          2205,  2062,     2,  4178,  6233,  4115, 31221,  4119,  4036,  2178,
          2113,  2522, 13197,  2116,  3838,  2079,  5550,  2069,  4651,  4538,
             2, 15259, 10455, 31221, 23731,  2259,  4295,  2522, 12263,   545,
          2079,  8262,  2470,  4740,  2069,  3691,  4538,     2,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [20]:
# ✅ TrainingArguments
training_args = TrainingArguments(
    output_dir="./global_results",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=5e-5,
    gradient_accumulation_steps=2,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
        
    load_best_model_at_end=True,
    metric_for_best_model='full_order_accuracy',
    greater_is_better=True,
    seed=42,
)

# ✅ Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# 학습 실행

In [ ]:
trainer.train()

In [17]:
# import shutil

# checkpoints = ["checkpoint-368", "checkpoint-736", "checkpoint-1104", "checkpoint-1472", "checkpoint-1840"]
# for ckpt in checkpoints:
#     shutil.rmtree(f"/kaggle/working/global_results/{ckpt}", ignore_errors=True)

In [ ]:
# ✅ best checkpoint 기준으로 모델 저장
save_path = "./global_results/best_model"

trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

# 튜닝

In [27]:
def run_global_tuning(train_dataset, val_dataset, tokenizer, n_trials=5):
    results_path = './global_results/tuning_log.csv'
    if os.path.exists(results_path):
        results = pd.read_csv(results_path).to_dict(orient='records')
        start_trial = len(results)
    else:
        results = []
        start_trial = 0

    for trial in range(start_trial, n_trials):
        print(f"\n🎯 Trial {trial + 1} 시작")
        lr = float(loguniform.rvs(2e-5, 5e-5))
        wd = float(loguniform.rvs(0.01, 0.07))
        epochs = int(np.random.randint(7, 13))
        batch_size = 16
        total_steps = (len(train_dataset) // batch_size) * epochs
        warmup = int(total_steps * np.random.uniform(0.02, 0.08))

        args = TrainingArguments(
            output_dir=f'./global_results/trial_{trial+1}',
            learning_rate=lr,
            weight_decay=wd,
            warmup_steps=warmup,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=64,
            num_train_epochs=epochs,
            gradient_accumulation_steps=1,
            lr_scheduler_type='linear',
            logging_dir='./roberta_logs',
            logging_steps=100,
            save_strategy="epoch",
            save_total_limit=2,
            eval_strategy='epoch',
            seed=42,
            load_best_model_at_end=True,
            metric_for_best_model='full_order_accuracy',
            greater_is_better=True,
            report_to='none',
            fp16=True,
            optim='adamw_torch_fused'
        )

        trainer = Trainer(
            model=GlobalOrderModel("klue/roberta-large"),
            args=args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )

        try:
            trainer.train()
            eval_results = trainer.evaluate()

            save_path = f'./global_results/trial_{trial+1}/best_model'

            try:
                trainer.save_model(save_path)
                tokenizer.save_pretrained(save_path)
                model_saved = True
            except Exception as e:
                print(f"⚠️ 모델 저장 실패: {e}")
                model_saved = False
                save_path = "FAILED"
                
            results.append({
                'trial': trial + 1,
                'learning_rate': lr,
                'weight_decay': wd,
                'warmup_steps': warmup,
                'epochs': epochs,
                'sentence_accuracy': eval_results.get('sentence_accuracy'),
                'full_order_accuracy': eval_results.get('full_order_accuracy'),
                'eval_loss': eval_results.get('eval_loss'),
                'model_saved': model_saved,
                'save_path': save_path
            })
            pd.DataFrame(results).to_csv(results_path, index=False)
            # trial 종료 후 checkpoint 자동 삭제 (save_total_limit 무력화 대비)
            import shutil
            output_dir = f'./global_results/trial_{trial+1}'
            for subdir in os.listdir(output_dir):
                if subdir.startswith("checkpoint"):
                    shutil.rmtree(os.path.join(output_dir, subdir), ignore_errors=True)


            print(f"✅ Trial {trial+1} | FullOrderAcc: {eval_results['full_order_accuracy']:.4f} | Loss: {eval_results['eval_loss']:.4f}")

        except Exception as e:
            print(f"⛔ Trial {trial+1} 중 오류 발생: {e}")
            break

    print("\n🏆 상위 Trial:")
    top_trials = pd.DataFrame(results).sort_values(by="full_order_accuracy", ascending=False).head(1)
    print(top_trials)
    return top_trials


In [ ]:
top_trials = run_global_tuning(train_dataset, val_dataset, tokenizer, n_trials=3)

In [ ]:
df = pd.read_csv('./global_results/tuning_log.csv')

for trial_num in [1, 2, 3]:
    model_path = f"./global_results/trial_{trial_num}/best_model"
    try:
        print(f"\n🔁 Trial {trial_num} 평가 중...")

        model = GlobalOrderModel("klue/roberta-large")
        model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin"), strict=False)

        tokenizer = AutoTokenizer.from_pretrained(model_path)
        training_args = TrainingArguments(
            output_dir=f"./eval_retry/trial_{trial_num}",  # 🔄 개별 폴더로 구분 추천
            per_device_eval_batch_size=64,
            report_to='none'  # optional: 로그 미기록 시
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        results = trainer.evaluate()
        print(f"✅ Trial {trial_num} 결과: {results}")

        # Trial이 이미 tuning_log.csv에 있으면 update
        if (df['trial'] == trial_num).any():
            idx = df[df['trial'] == trial_num].index[0]
            df.loc[idx, 'sentence_accuracy'] = results.get('sentence_accuracy')
            df.loc[idx, 'full_order_accuracy'] = results.get('full_order_accuracy')
            df.loc[idx, 'eval_loss'] = results.get('eval_loss')
        else:
            df = pd.concat([
                df,
                pd.DataFrame([{
                    'trial': trial_num,
                    'learning_rate': None,
                    'weight_decay': None,
                    'warmup_steps': None,
                    'epochs': None,
                    'sentence_accuracy': results.get('sentence_accuracy'),
                    'full_order_accuracy': results.get('full_order_accuracy'),
                    'eval_loss': results.get('eval_loss'),
                    'model_saved': True,
                    'save_path': model_path
                }])
            ], ignore_index=True)

    except Exception as e:
        print(f"❌ Trial {trial_num} 평가 실패: {e}")

df.to_csv('./global_results/tuning_log.csv', index=False)


In [ ]:
os.listdir("./global_results/trial_2/best_model")

In [ ]:
# ✅ tuning 결과에서 custom_score 기반으로 새로 평가
# ✅ 모델 저장된 trial만 남김
df = df[df["model_saved"] == True]

# ✅ 너무 손실이 큰 trial은 제거 (불안정하거나 과적합 위험)
df = df.query("eval_loss < 1.0") 

# ✅ custom_score 계산: 정확도는 높을수록, 손실은 낮을수록 좋음
df["custom_score"] = df["full_order_accuracy"] - 0.5 * df["eval_loss"]

# ✅ 기존 full accuracy 기준은 유지
top_trial_custom = df.sort_values("custom_score", ascending=False).iloc[0]

# ✅ best model 경로 정의 (새 방식)
best_model_path = top_trial_custom["save_path"]
print(f"🏆 새로운 방식 기준 Best Model 경로: {best_model_path}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ GlobalOrderModel은 이미 정의돼 있다고 가정
model = GlobalOrderModel("klue/roberta-large")

# ✅ safetensors 파일로부터 로드
state_dict = load_file(f"{best_model_path}/model.safetensors")
model.load_state_dict(state_dict)
model.to(device)
model.eval()


In [40]:
import shutil

final_model_dir = "./global_results/best_model_custom"
shutil.copytree(best_model_path, final_model_dir, dirs_exist_ok=True)
print(f"📦 최종 best model 저장됨: {final_model_dir}")


📦 최종 best model 저장됨: ./global_results/best_model_custom


# 추론

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# best_model_path = "./global_results/best_model_custom"  # ← 경로 꼭 이걸로 맞춰주세요
best_model_path = "./global_results/best_model"  # ← 경로 꼭 이걸로 맞춰주세요

model = GlobalOrderModel(
    model_name="klue/roberta-large",
    tokenizer=tokenizer  # tokenizer 전달
)
state_dict = load_file(f"{best_model_path}/model.safetensors")
model.load_state_dict(state_dict)
model.to(device)
model.eval()

In [55]:
# inference.py
def predict(model, test_df, tokenizer, device, batch_size=32):
    test_dataset = GlobalOrderDataset(test_df, tokenizer, labels=None)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            logits = model(input_ids, attention_mask)['logits']  # ✅ dict에서 'logits' 꺼냄
            preds = logits.argmax(-1).cpu().numpy()
            all_preds.append(preds)
    all_preds = np.concatenate(all_preds, axis=0)
    # [문장0은 몇 번째, ...] → [answer_0, answer_1, ...]로 역변환
    answers = []
    for row in all_preds:
        answer = [0]*4
        for sent_idx, pos in enumerate(row):
            answer[pos] = sent_idx
        answers.append(answer)
    return np.array(answers)

def save_submission(test_df, answers, submission_path, output_path):
    sub = pd.read_csv(submission_path)
    for i in range(4):
        sub[f'answer_{i}'] = answers[:, i]
    sub.to_csv(output_path, index=False)


# 예측 및 저장

In [ ]:
# 예측 수행
answers = predict(
    model=model,
    test_df=test_df,
    tokenizer=tokenizer,
    device=device,
    batch_size=32
)

# 제출 파일 저장
save_submission(
    test_df=test_df,
    answers=answers,
    submission_path="./sample_submission.csv",
    output_path="./submission.csv"
)

print("✅ submission.csv 저장 완료")